In [14]:
# importing libraries 
import requests
import json
from datetime import datetime, timedelta
import pandas as pd

In [15]:
# using the airqo api to get device details. such as name, status, whether airqo or not
response_API = requests.get("https://platform.airqo.net/api/v1/devices?tenant=airqo&network=airqo")
output_test = json.dumps(response_API.json(),sort_keys=True,indent =4)
device_details = json.loads(output_test)

In [16]:
# function to get device details 
def get_device_details(): 
    for device in device_data:
        device_data = []
        status = device.get("isActive",False)
        network = device.get("network",False)
        if network == "airqo": 
            device_data.append({"name":device["name"]})
    return device_data

In [17]:
def time_window_calculation (): # goal is to have uptime readings count every 60 seconds 
    end_date_time = datetime.utcnow()
    start_date_time = end_date_time - timedelta(minutes = 60)
    return start_date_time,end_date_time

In [18]:
def data_collection(): 
    api_data = []
    airqo_device_data = []
    [x,y] = time_window_calculation()
    hourly_data = get_device_details()
    for device in hourly_data: # api doesnt return data for PM10 to factor it in the calculation
        url = f'https://platform.airqo.net/api/v1/monitor/devices/uptime?' \
        f'tenant=airqo&' \
        f'startDate={datetime.strftime(x, "%Y-%m-%dT%H:%M:%S.%fZ")}&' \
        f'endDate={datetime.strftime(y, "%Y-%m-%dT%H:%M:%S.%fZ")}&' \
        f'device_name={device["name"]}'
        response = requests.get(url).json()
        api_data.extend(response["data"])

        for record in api_data:
            for value in record["values"]:
                airqo_device_data.append({"device":value["device_name"],
                                         "s1_pm2_5":value["sensor_ome_pm2_5"],
                                         "s2_pm2_5": value["sensor_two_pm2_5"]})
    airqo_device_data_df = pd.DataFrame(airqo_device_data)
    cleaned_device_data = airqo_device_data_df.dropna(subset = ["s1_pm2_5","s2_pm2_5"])

    return cleaned_device_data


In [19]:
def calculate_uptime():
    expected_data_count  = 60 #60 is just an assumption number to try out the calculation 
    actual_count_per_device = data_collection().groupyby(pd.Grouper(key ="device")).count()
    actual_count_per_device.reset_index(inplace=True)
    actual_count_per_device["uptime"] = (actual_count_per_device["s2_pm2_5"]/expected_data_count)*100
    # using a single coloumn to check for uptime, because i assume we have already dropped all rows with funny data

    return actual_count_per_device 